## Pytorch Tutorials : Auotograd
- [Pytorch Tutorials](https://pytorch.org/tutorials/)
- [Auotograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

### Automatic Differentiation(자동 미분) with torch.autograd
신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 역전파로,  매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수 의 기울기에 따라 조정됩니다 .

이러한 그라디언트를 계산하기 위해 PyTorch에는 `torch.autograd`가 내장되어 있으며, 모든 계산 그래프에 대한 기울기 자동 계산을 지원합니다. 입력으로 간단한 하나의 층 신경망을 고려 x파라미터 w와 b, 일부 손실 함수. 다음과 같은 방식으로 PyTorch에서 정의할 수 있습니다.

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

### Tensors, Functions and Computational graph
![](https://pytorch.org/tutorials/_images/comp-graph.png)
다음과 같은 계산 그래프를 정의할 수 있음.

이 네트워크에서 w와 b는 최적화해야 하는 매개변수이므로, 이러한 변수와 관련하여 손실 함수의 그래디언트를 계산할 수 있어야 한다. 이를 위해 텐서의 `required_grad` 속성을 설정합니다.(텐서를 생성할 때 또는 나중에 `x.required_grad_(True)` 메서드를 사용하여 `required_grad` 값을 설정할 수 있습니다.)

계산 그래프를 구성하기 위해 텐서에 적용하는 함수는 클래스 함수의 객체로, 함수를 순전파로 계산하는 방법과 역전파 단계에서 파생 함수를 계산하는 방법을 알고 있습니다.

In [2]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f5b5f0189d0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f5b5f018a50>


### Computing Gradients
신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실 함수의 도함수를 계산해야 함.  

이러한 도함수를 계산하기 위해 `loss.backward()`을 선언하고, w.grad 및 b.grad에서 값을 계산한다.

> 성능상의 이유로 주어진 그래프에서 `backward`를 한 번만 사용하여 그레이디언트 계산을 수행할 수 있습니다. 같은 그래프에서 여러 번의 `backward`를 해야 한다면 `retain_graph=True`을 `backward` 넘겨야한다.

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0352, 0.0399, 0.2644],
        [0.0352, 0.0399, 0.2644],
        [0.0352, 0.0399, 0.2644],
        [0.0352, 0.0399, 0.2644],
        [0.0352, 0.0399, 0.2644]])
tensor([0.0352, 0.0399, 0.2644])


### Disabling Gradient Tracking
기본적으로 `retain_graph=True`가 있는 모든 텐서는 계산 이력을 추적하고 그래디언트 계산을 지원한다. 그러나 예를 들어 모델을 교육하고 일부 입력 데이터에만 적용하려는 경우, 즉 forward 연산만 수행하려는 경우처럼 그렇게 할 필요가 없는 경우가 있습니다. 연산 코드를 `torch.no_grad` 블록으로 둘러싸면 연산 추적을 중지할 수 있다.

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


`detach()`를 통해 동일한 결과를 반환할 수 있다.

In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


Disabling Gradient Tracking 사용 이유
 1. 신경망의 일부 매개 변수를 동결 - 사전훈련된 신경망을 finetuning하기 위함
 2. 그래디언트를 추적하지 않는 tensor에 대한 계산이 더욱 효율적이기 때문에 순전파 계산만 하는 경우 연산 속도를 높일 수 있다.